In [1]:
import pyspark 
from pyspark import SparkContext 
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('yarn').appName('').getOrCreate()  

In [14]:
census = spark.read.csv("/example/census2015_1.csv", header = True)
census.createOrReplaceTempView("census")

In [15]:
census.count()

927843

In [16]:
census.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)
 |-- V11: string (nullable = true)
 |-- V12: string (nullable = true)
 |-- V13: string (nullable = true)
 |-- V14: string (nullable = true)
 |-- V15: string (nullable = true)
 |-- V16: string (nullable = true)
 |-- V17: string (nullable = true)
 |-- V18: string (nullable = true)
 |-- V19: string (nullable = true)
 |-- V20: string (nullable = true)
 |-- V21: string (nullable = true)
 |-- V22: string (nullable = true)
 |-- V23: string (nullable = true)
 |-- V24: string (nullable = true)
 |-- V25: string (nullable = true)
 |-- V26: string (nullable = true)
 |-- V27: string (nullable = true)
 |-- V28: string (nullab

In [22]:
spark.sql("select distinct V2 from census").show()

+---+
| V2|
+---+
| 11|
| 29|
| 34|
| 22|
| 35|
| 31|
| 26|
| 23|
| 38|
| 25|
| 33|
| 24|
| 32|
| 36|
| 37|
| 39|
| 21|
+---+



In [25]:
results = spark.sql("select V2 as code, V57, count(*) as cnt from census group by V2, V57").toPandas()
results

,code,V57,cnt
0,26,0,19455
1,31,1,1762
2,26,1,324
3,34,0,43562
4,37,0,59496
5,36,1,528
6,29,0,5147
7,39,1,59
8,32,0,35817
9,33,1,330


In [26]:
result1 = results.loc[results['V57']=='1']
result2 = results.loc[results['V57']=='0']

In [32]:
import pandas as pd

cityCodes = pd.read_csv('~/hadoop_example/censusCodes.csv', sep=',', dtype={'code':object})

In [44]:
result = pd.merge(result1, result2, how = 'left', on = 'code')
result

,code,V57_x,cnt_x,V57_y,cnt_y
0,31,1,1762,0,204306
1,26,1,324,0,19455
2,36,1,528,0,46920
3,39,1,59,0,12936
4,33,1,330,0,32567
5,11,1,1499,0,152322
6,23,1,480,0,47702
7,38,1,940,0,65072
8,22,1,493,0,42927
9,24,1,196,0,26029


In [45]:
result['ratio'] = result['cnt_x'] * 100 / (result['cnt_x'] + result['cnt_y'])
result

,code,V57_x,cnt_x,V57_y,cnt_y,ratio
0,31,1,1762,0,204306,0.855058
1,26,1,324,0,19455,1.638101
2,36,1,528,0,46920,1.112797
3,39,1,59,0,12936,0.454021
4,33,1,330,0,32567,1.003131
5,11,1,1499,0,152322,0.974509
6,23,1,480,0,47702,0.996223
7,38,1,940,0,65072,1.423984
8,22,1,493,0,42927,1.135421
9,24,1,196,0,26029,0.747378


In [46]:
result = pd.merge(result, cityCodes, how='left', on='code')
result

,code,V57_x,cnt_x,V57_y,cnt_y,ratio,cityName
0,31,1,1762,0,204306,0.855058,경기도
1,26,1,324,0,19455,1.638101,울산광역시
2,36,1,528,0,46920,1.112797,전라남도
3,39,1,59,0,12936,0.454021,제주도
4,33,1,330,0,32567,1.003131,충청북도
5,11,1,1499,0,152322,0.974509,서울특별시
6,23,1,480,0,47702,0.996223,인천광역시
7,38,1,940,0,65072,1.423984,경상남도
8,22,1,493,0,42927,1.135421,대구광역시
9,24,1,196,0,26029,0.747378,광주광역시
